In [73]:
import io
import json
import os
from collections import OrderedDict
from shutil import copyfile
import xml.etree.cElementTree as ET

import pandas as pd

import nltk

In [7]:
old_nltk_data = nltk.data.path[0]

new_nltk_data = "packages/"

# ABC Corpus


In [86]:
# ABC corpus.
directory = new_nltk_data+'/corpora/abc/'
if not os.path.exists(directory):
    os.makedirs(directory)

with io.open(old_nltk_data+'/corpora/abc/rural.txt') as fin:
    rural_texts = [line.strip() for line in fin if line.strip()]
with io.open(old_nltk_data+'/corpora/abc/science.txt', encoding='latin_1') as fin:
    science_texts = [line.strip().encode('utf8').decode('utf8') for line in fin if 
                    line.strip().encode('utf8').decode('utf8')]

rural_df = pd.DataFrame({'text':rural_texts})
rural_df['subcorpora'] = 'Rural News'

science_df = pd.DataFrame({'text':science_texts})
science_df['subcorpora'] = 'Science News'

df_abc = pd.concat([rural_df, science_df])
df_abc.to_csv(new_nltk_data+'/corpora/abc/abc.tsv', sep='\t', index=False)
df_abc = pd.read_csv(new_nltk_data+'/corpora/abc/abc.tsv', sep='\t', 
                     dtype={'text':str, 'subcorpora':str})

abc_meta = {'title':'Australian Broadcasting Commission 2006',
            'source': 'http://www.abc.net.au/',
            'subcorpora': {'Rural News': {'source': 'http://www.abc.net.au/rural/news/'},
                           'Science News': {'source': 'http://www.abc.net.au/science/news/'}
                          },
             'xml': {'id':'abc', 'name':"Australian Broadcasting Commission 2006",
                     'webpage':"http://www.abc.net.au/", 'author':"Australian Broadcasting Commission",
                      'unzip':"1"}
           }


abc_xml = ET.Element("package", id="abc", name="Australian Broadcasting Commission 2006",
                  webpage="http://www.abc.net.au/", author="Australian Broadcasting Commission",
                  unzip="1")
tree = ET.ElementTree(root)
tree.write(new_nltk_data+'/corpora/abc/abc.xml')

with open(new_nltk_data+'/corpora/abc/abc-meta.json', 'w') as fout:
    json.dump(abc_meta, fout, sort_keys=True, indent=4, separators=(',', ': '))
    

In [87]:
df_abc.head()

,text,subcorpora
0,PM denies knowledge of AWB kickbacks,Rural News
1,The Prime Minister has denied he knew AWB was ...,Rural News
2,Letters from John Howard and Deputy Prime Mini...,Rural News
3,In one of the letters Mr Howard asks AWB manag...,Rural News
4,The Opposition's Gavan O'Connor says the lette...,Rural News


# Brown Corpus

In [85]:
# Brown

directory = new_nltk_data+'/corpora/brown/'
if not os.path.exists(directory):
    os.makedirs(directory)
    
    
with open(old_nltk_data+'/corpora/brown/cats.txt') as fin:
     categories = {line.strip().split(' ')[0]:line.strip().split(' ')[1] 
                   for line in fin}
        
brown_dir = old_nltk_data+'/corpora/brown/'

rows = []
for filename in os.listdir(brown_dir):
    if filename in ['CONTENTS', 'cats.txt', 'README']:
        continue
    cat = categories[filename]
    with open(brown_dir+filename) as fin:
        i = -1
        for paragraph in fin.read().split('\n\n'):
            if not paragraph.strip():
                continue
            i += 1
            j = -1
            for sent in paragraph.split('\n'):
                if not sent.strip():
                    continue
                j += 1
                raw = sent.strip()
                text, pos = zip(*[word.split('/') for word in raw.split()])
                rows.append({'filename': filename, 
                              'para_id': i, 
                              'sent_id': j, 
                              'raw_text': raw, 
                              'tokenized_text': ' '.join(text), 
                              'tokenized_pos': ' '.join(pos), 
                              'label': cat})
                
                
df_brown = pd.DataFrame(rows)[['filename', 'para_id', 'sent_id', 
                              'raw_text', 'tokenized_text', 'tokenized_pos', 'label']]
df_brown.to_csv(new_nltk_data+'/corpora/brown/brown.tsv', sep='\t', index=False)

df_brown = pd.read_csv(new_nltk_data+'/corpora/brown/brown.tsv', sep='\t', 
                     dtype={'filename':str, 'para_id':int, 'sent_id':int,
                             'raw_text':str, 'tokenized_text':str, 'tokenized_pos':str,
                           'label':str})

df_brown_cats = df_brown[['filename', 'label']].drop_duplicates().sort_values('filename')
df_brown_cats.to_csv(new_nltk_data+'/corpora/brown/cats.tsv', sep='\t', index=False)

brown_readme = """BROWN CORPUS

A Standard Corpus of Present-Day Edited American
English, for use with Digital Computers.

by W. N. Francis and H. Kucera (1964)
Department of Linguistics, Brown University
Providence, Rhode Island, USA

Revised 1971, Revised and Amplified 1979

http://www.hit.uib.no/icame/brown/bcm.html

Distributed with the permission of the copyright holder,
redistribution permitted."""

brown_meta = {'title':'Brown Corpus',
              'description': str('A Standard Corpus of Present-Day Edited American English, '
                               'for use with Digital Computers.'),
              'authors': 'W. N. Francis and H. Kucera (1964)',
              'url': 'http://www.hit.uib.no/icame/brown/bcm.html',
              'readme': brown_readme}

with open(new_nltk_data+'/corpora/brown/brown-meta.json', 'w') as fout:
    json.dump(brown_meta, fout, sort_keys=True, indent=4, separators=(',', ': '))

In [89]:
df_brown.head()

,filename,para_id,sent_id,raw_text,tokenized_text,tokenized_pos,label
0,cd05,0,0,"Furthermore/rb ,/, as/cs an/at encouragement/n...","Furthermore , as an encouragement to revisioni...","rb , cs at nn in nn nn , pps rb bez jj to vb c...",religion
1,cd05,0,1,The/at Unitarian/jj clergy/nns were/bed an/at ...,The Unitarian clergy were an exclusive club of...,at jj nns bed at jj nn in vbn nns -- cs at nn ...,religion
2,cd05,0,2,"Ezra/np Stiles/np Gannett/np ,/, an/at honorab...","Ezra Stiles Gannett , an honorable representat...","np np np , at jj nn in at nn , vbd ppl rb in a...",religion
3,cd05,0,3,"Even/rb so/rb ,/, Gannett/np judiciously/rb ar...","Even so , Gannett judiciously argued , the Ass...","rb rb , np rb vbd , at nn-tl md rb vb cs np ``...",religion
4,cd05,0,4,We/ppss today/nr are/ber not/* entitled/vbn to...,We today are not entitled to excoriate honest ...,ppss nr ber * vbn to vb jj nns wps vbd np to b...,religion


# Gazetteers

In [96]:
# Gazetteers

directory = new_nltk_data+'/corpora/gazetteers/'
if not os.path.exists(directory):
    os.makedirs(directory)
    
gazetteers_filename2labels = {'mexstates.txt':'Mexico States',
                              'caprovinces.txt': 'Canada Provinces',
                              'usstateabbrev.txt': 'US State Abbreviations',
                              'uscities.txt': 'US Cities',
                              'countries.txt': 'Countries',
                              'isocountries.txt': 'Countries ISO codes',
                              'nationalities.txt': 'Nationalities',
                              'usstates.txt': 'US States'
                             }

rows = []
for filename in os.listdir(old_nltk_data+'/corpora/gazetteers/'):
    if filename in ['LICENSE.txt']:
        continue
    label = gazetteers_filename2labels[filename]
    with io.open(old_nltk_data+'/corpora/gazetteers/'+filename, encoding='ISO-8859-2') as fin:
        for line in fin:
            if line.strip():
                text = line.strip()
                if text == 'QuerĂŠtaro':
                    text = 'Querétaro'
                rows.append({'text':text, 'label':label})

df_gazetteers = pd.DataFrame(rows)[['text', 'label']]

#alpabet = list('abcdefghijklmnopqrstuvwxyz. ()-,') + list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
#alpabet += ["'"]
#[word for word in df_gazetteers['text'] if any(ch for ch in word if ch not in alpabet)]

df_gazetteers.to_csv(new_nltk_data + '/corpora/gazetteers/gazetteers.tsv', sep='\t', index=False)
df_gazetteers = pd.read_csv(new_nltk_data + '/corpora/gazetteers/gazetteers.tsv', sep='\t', 
                     dtype={'text':str, 'label':str})

gazetteers_filename2labels = {'mexstates.txt':'Mexico States',
                              'caprovinces.txt': 'Canada Provinces',
                              'usstateabbrev.txt': 'US State Abbreviations',
                              'uscities.txt': 'US Cities',
                              'countries.txt': 'Countries',
                              'isocountries.txt': 'Countries ISO codes',
                              'nationalities.txt': 'Nationalities',
                              'usstates.txt': 'US States'
                             }

gazetteers_meta = {'title':'Geolocation Gazeteers',
                    'subcorpora': {'Mexico States': {'original_file': 'mexstates.txt'},
                                   'Canada Provinces': {'original_file': 'caprovinces.txt'},
                                   'US State Abbreviations': {'original_file': 'usstates.txt'},
                                   'US States': {'original_file': 'usstateabbrev.txt'},
                                   'US Cities': {'original_file':'uscities.txt',
                                                 'source': 'http://en.wikipedia.org/wiki/List_of_cities_in_the_United_States_with_over_100%2C000_people',
                                                 'license': 'GNU Free Documentation License',
                                                 'license_url': 'http://www.gnu.org/copyleft/fdl.html'
                                                },
                                   'Countries': {'original_file':'countries.txt',
                                                 'source':'http://en.wikipedia.org/wiki/List_of_countries',
                                                 'license': 'GNU Free Documentation License',
                                                 'license_url': 'http://www.gnu.org/copyleft/fdl.html'
                                                },
                                   'Countries ISO codes': {'original_file': 'isocountries.txt',
                                                          'source': 'http://www.guavastudios.com/country-list.htm'
                                                          },
                                   'Nationalities': {'original_file': 'nationalities.txt',
                                                    'source': 'http://www.guavastudios.com/nationalities-list.htm'
                                                    },
                                  }
                    }

with open(new_nltk_data+'/corpora/gazetteers/gazetteers-meta.json', 'w') as fout:
    json.dump(gazetteers_meta, fout, sort_keys=True, indent=4, separators=(',', ': '))

# Words

In [98]:
# Words

directory = new_nltk_data+'/corpora/words/'
if not os.path.exists(directory):
    os.makedirs(directory)
    
en_words = []
with open(old_nltk_data+'/corpora/words/en') as fin:
    for line in fin:
        en_words.append(line.strip())

basic_en_words = []
with open(old_nltk_data+'/corpora/words/en-basic') as fin:
    for line in fin:
        basic_en_words.append(line.strip())
        
words_meta = {'title':'Word Lists',
              'subcorpora': {'Unix Words':{'source':'http://en.wikipedia.org/wiki/Words_(Unix)'},
                           'Ogden Basic English': {'title': 'The ABC of Basic English',
                                                   'author':'C.K. Ogden (1932)'}
                          }
            }

unix_words = pd.DataFrame({'text':en_words})
ogden_words = pd.DataFrame({'text':basic_en_words})

unix_words.to_csv(new_nltk_data + '/corpora/words/unix_words.tsv', sep='\t', index=False)
ogden_words.to_csv(new_nltk_data + '/corpora/words/ogden_words.tsv', sep='\t', index=False)

unix_words = pd.read_csv(new_nltk_data + '/corpora/words/unix_words.tsv', sep='\t', dtype={'text':str})
ogden_words = pd.read_csv(new_nltk_data + '/corpora/words/ogden_words.tsv', sep='\t', dtype={'text':str})

with open(new_nltk_data+'/corpora/words/words-meta.json', 'w') as fout:
    json.dump(words_meta, fout, sort_keys=True, indent=4, separators=(',', ': '))

# Movie Review

In [107]:

directory = new_nltk_data+'/corpora/movie_reviews/'
if not os.path.exists(directory):
    os.makedirs(directory)
    
rows = []

for filename in sorted(os.listdir(old_nltk_data+'/corpora/movie_reviews/pos/')):
    fold, html_id = filename[:-4].split('_')
    fold_id = int(int(fold[2:]) / 100)
    
    with open(old_nltk_data+'/corpora/movie_reviews/pos/'+filename) as fin:
        for sent_id, line in enumerate(fin):
                rows.append({'fold_id':fold_id, 
                             'cv_tag':fold, 
                             'html_id':html_id, 
                             'sent_id':sent_id, 
                             'text':line.strip(),
                             'tag':'pos'
                            })
                
for filename in sorted(os.listdir(old_nltk_data+'/corpora/movie_reviews/neg/')):
    fold, html_id = filename[:-4].split('_')
    fold_id = int(int(fold[2:]) / 100)

    with open(old_nltk_data+'/corpora/movie_reviews/neg/'+filename) as fin:
        for sent_id, line in enumerate(fin):
                rows.append({'fold_id':fold_id, 
                             'cv_tag':fold, 
                             'html_id':html_id, 
                             'sent_id':sent_id, 
                             'text':line.strip(),
                             'tag':'neg'
                            })
                
df_movie_reivews = pd.DataFrame(rows)[['fold_id', 'cv_tag', 'html_id', 'sent_id', 'text', 'tag']]

df_movie_reivews.to_csv(new_nltk_data + '/corpora/movie_reviews/movie_review.tsv', sep='\t', index=False)
df_movie_reivews = pd.read_csv(new_nltk_data + '/corpora/movie_reviews/movie_review.tsv', sep='\t', 
                     dtype={'fold_id':int, 'cv_tag':str, 'html_id':str, 'sent_id':int,
                            'text':str, 'tag':str})


mr_bibtext = """@InProceedings{Pang+Lee:04a,
  author =       {Bo Pang and Lillian Lee},
  title =        {A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts},
  booktitle =    "Proceedings of the ACL",
  year =         2004
}"""



mr_meta = {'title': 'Sentiment Polarity Dataset Version 2.0',
           'aka': 'Moview Review Data',
           'source': 'http://www.cs.cornell.edu/people/pabo/movie-review-data/',
           'authors': 'Bo Pang and Lillian Lee',
           'license': 'Distributed with NLTK with permission from the authors.',
            'bibtex':mr_bibtext}

with open(new_nltk_data+'/corpora/movie_reviews/movie_reviews-meta.json', 'w') as fout:
    json.dump(mr_meta, fout, sort_keys=True, indent=4, separators=(',', ': '))